# Домашнее задание. Титаник
В этом домашнем задании вам предлагается решить задачу предсказания выживших пассажиров Титаника. Эту задачу машинного обучения рано или поздно решает, наверное, любой, кто обучается анализу данных. Информация о датасете доступна по <a href=https://www.kaggle.com/c/titanic>ссылке</a>.

Кстати, здесь же есть много готовых решений нашей задачи. Решение можно посмотреть в разделе <a href=https://www.kaggle.com/c/titanic/code>code на kaggle</a>.

Некоторые решения, которые уже есть:
https://www.kaggle.com/hacktech33/titanic-solution-xgboost (простое решение)

https://www.kaggle.com/shrutijhaa/in-top-3-titanic-machine-learning-from-disaster (возможно, слишком много визуализации, но хорошее качество)

https://www.kaggle.com/blackhurt/top-3-using-voting-classifier (сильное решение, использует метод простого голосования трёх сильных моделей)

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

## Загрузка данных

Загрузим датасет Titanic из приложенных файлов. В первом файле находится информация о пассажирах, во втором -- информация о том, выжил пассажир или нет.

Если вы запускаете ноутбук на Google Colab, самым простым способом загрузить данные в ноутбук будет примонтировать к Colab ваш Google Drive. Для этого сначала положите в свою корневую директорию Google Drive  файлы, приложенные к этому ноутбуку, а затем выполните три закоментированные строчки в клетке ниже. <a href=https://www.machinelearningmastery.ru/downloading-datasets-into-google-drive-via-google-colab-bcb1b30b0166/>Подробная инструкция по работе с Google Drive + Google Colab</a>

In [ ]:
#from google.colab import drive
#drive.mount('/content/gdrive/')
#data = pd.read_csv('/content/gdrive/My Drive/titanic_data.csv, index_col='PassengerId')

#Если запускаете ноутбук локально:
data = pd.read_csv('titanic_data.csv', index_col='PassengerId')

In [ ]:
basic_features = data.columns

y = pd.read_csv('titanic_surv.csv')
y.index = data.index

print(f'Всего {len(data)} пассажиров в выборке')
data.head()

Всего 891 пассажиров в выборке


,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
PassengerId,,,,,,,,,,
1,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
2,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
3,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
4,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
5,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [ ]:
y.head()

,Survived
PassengerId,
1,0
2,1
3,1
4,1
5,0


In [ ]:
data = data.join(y)
data.head()

,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Survived
PassengerId,,,,,,,,,,,
1,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S,0
2,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C,1
3,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,1
4,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S,1
5,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S,0


## Исследование датасета

### Задание 1 (1 балл)
Опишите датасет. Сколько в нём мужчин, сколько женщин? Посчитайте распределение по классам пассажиров. Используйте функцию ``pd.Series.value_counts``. <a href=https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.value_counts.html>Пример использования</a>

Посчитайте долю выживших мужчин к общему количеству мужчин, а также женщин к общему количеству женщин. Сделайте выводы. 
Проведите схожую аналитику по возрасту выживших и по классу выживших. 

Вам поможет функция ``plt.hist()``. <a href=https://matplotlib.org/3.3.1/gallery/pyplots/pyplot_text.html#sphx-glr-gallery-pyplots-pyplot-text-py> Пример использования </a> 

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

data = pd.read_csv('titanic.csv')

gender_counts = data['Sex'].value_counts()
print(gender_counts)

class_counts = data['Pclass'].value_counts()
print(class_counts)

survived_gender = data[data['Survived'] == 1]['Sex'].value_counts()
total_gender = data['Sex'].value_counts()
survival_rate_men = survived_gender['male'] / total_gender['male']
survival_rate_women = survived_gender['female'] / total_gender['female']

print(f"{survival_rate_men:.2%}")
print(f"{survival_rate_women:.2%}")

plt.hist(data[data['Survived'] == 1]['Age'].dropna(), bins=20, alpha=0.5, label='Выжившие')
plt.hist(data[data['Survived'] == 0]['Age'].dropna(), bins=20, alpha=0.5, label='Не выжившие')
plt.xlabel('Возраст')
plt.ylabel('Количество пассажиров')
plt.legend()
plt.show()

survived_class_counts = data[data['Survived'] == 1]['Pclass'].value_counts()
plt.bar(survived_class_counts.index, survived_class_counts.values)
plt.xlabel('Класс')
plt.ylabel('Количество выживших')
plt.show()


## Заполнение пропусков в данных
В данных имеются пропуски в трёх колонках: 

In [ ]:
data.columns[data.isna().any()].tolist()

['Age', 'Cabin', 'Embarked']

Для простоты заполним все пропуски в категориальных колонках новым классом "0", а в числовой колонке Age --- медианным значением. 

In [ ]:
data.loc[:, ['Cabin', 'Embarked']] = data.loc[:, ['Cabin', 'Embarked']].fillna('0')
data['Age'] = data['Age'].fillna(data['Age'].median())

## Подготовка плана тестирования

### Задание 2. (0 баллов)
Разделите выборку на обучающее и тестовое множество в соотношении 70:30.

In [ ]:
from sklearn.model_selection import train_test_split

data_train, data_test = train_test_split(data, test_size=0.3, random_state=42)

## Выделение новых признаков
Теперь отложим тестовые данные и приступим к моделированию. Внимательно изучите данные. Можете ли вы выделить признаки, которые не указаны явно в таблице?

### Задание 3 (1 балл)
 Сформируйте по крайней мере один новый признак и объясните ваш выбор. Пример признака сформирован за вас. 

Указания: 
- Пассажиров можно поделить на несколько классов по их именам.
- Различных номеров кают слишком много. Но буквы в номерах кают указывают на их местоположение. (***Этот признак сформирован в примере***)
- Возможно, имеет смысл отделить мальчиков от мужчин. Отделять девочек от женщин может быть не так важно
- Другие идеи для признаков можно почерпнуть на форуме в обсуждении задачи на kaggle.com.

In [ ]:
data

,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Survived
PassengerId,,,,,,,,,,,
1,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S,0
2,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C,1
3,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,1
4,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S,1
5,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S,0
...,...,...,...,...,...,...,...,...,...,...,...
887,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S,0
888,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S,1
889,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S,0


In [ ]:
data['Cabin']

PassengerId
1         0
2       C85
3         0
4      C123
5         0
       ... 
887       0
888     B42
889       0
890    C148
891       0
Name: Cabin, Length: 891, dtype: object

In [ ]:
def get_cabin_letter(row):
    return row['Cabin'][0]

data_train['cabin_type'] = data_full.apply(get_cabin_letter, axis=1)

Сюда добавьте описание вашего нового признака

In [ ]:
def compute_name_length(row):
    return len(row['Name'])

data_full['NameLength'] = data_full.apply(compute_name_length, axis=1)


,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Survived,cabin_type
PassengerId,,,,,,,,,,,,
1,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,0,S,0,0
2,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C,1,C
3,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,0,S,1,0
4,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S,1,C
5,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,0,S,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
887,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,0,S,0,0
888,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S,1,B
889,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,28.0,1,2,W./C. 6607,23.4500,0,S,0,0


## Кодирование категориальных признаков и обработка данных
Все признаки сейчас делятся на числовые, бинарные, категориальные и текстовые. К текстовым признакам можно отнести имя пассажира, номер билета и номер каюты: значения этих признаков уникальны почти для всех пассажиров, и простого способа использовать их в модели не существует.

### Задание 4 (0 баллов)
Пока используя только изначальные признаки, перекодируйте категориальные признаки функцией ``pd.get_dummies``. Естественно, имена пассажиров и номера билетов и кают кодировать не стоит, поскольку все значения этих признаков уникальные. 

In [ ]:
data_full_encoded = pd.get_dummies(data_full, columns=['Sex', 'Embarked', 'Pclass'], drop_first=True)


## Обучение baseline-модели

### Задание 5 (1 балл)

Обучите одну из простых известных вам моделей. Измерьте качество полученной модели на кросс-валидации (используйте только обучающую выборку!)

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score

X_train = data_train.drop('Survived', axis=1)
y_train = data_train['Survived']

model = LogisticRegression(max_iter=200)
cv_scores = cross_val_score(model, X_train, y_train, cv=5)

print(f"Кросс-валидация: средняя точность = {cv_scores.mean():.2f}, стандартное отклонение = {cv_scores.std():.2f}")


### Задание 6 (1 балл)
Добавьте в модель два новых сгенерированных признака. Если нужно, добавьте признаки, сгенерированные ohe-hot кодированием. Обучите ту же самую модель на расширенном множестве признаков. Улучшилось ли качество предсказания на кросс-валидации?

In [ ]:
# Добавляем новые признаки
data_train['CabinLetter'] = data_train['Cabin'].apply(lambda x: str(x)[0] if pd.notna(x) else 'Unknown')
data_train['NameLength'] = data_train['Name'].apply(len)

# Кодируем категориальные признаки с использованием one-hot кодирования
data_train_encoded = pd.get_dummies(data_train, columns=['Sex', 'Embarked', 'Pclass', 'CabinLetter'], drop_first=True)

# Разделяем данные на X и y
X_train_expanded = data_train_encoded.drop('Survived', axis=1)
y_train_expanded = data_train_encoded['Survived']

# Обучаем модель логистической регрессии
model_expanded = LogisticRegression(max_iter=200)
cv_scores_expanded = cross_val_score(model_expanded, X_train_expanded, y_train_expanded, cv=5)

print(f"Кросс-валидация на расширенном множестве: средняя точность = {cv_scores_expanded.mean():.2f}, стандартное отклонение = {cv_scores_expanded.std():.2f}")


## Дополнительное моделирование
### Задание 7 (2 балла)
Теперь более серьёзно подойдём к моделированию. Попробуйте несколько алгоритмов из тех, что мы проходили в курсе. Вам помогут ноутбуки с линейными алгоритмами и выбором модели. Хотя бы для одного алгоритма проведите подбор оптимального гиперпараметра. 

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import GridSearchCV

# Разделяем данные на X и y
X_train = data_train_encoded.drop('Survived', axis=1)
y_train = data_train_encoded['Survived']

# Логистическая регрессия
log_model = LogisticRegression(max_iter=200)
log_cv_scores = cross_val_score(log_model, X_train, y_train, cv=5)
print(f"Логистическая регрессия: средняя точность = {log_cv_scores.mean():.2f}, стандартное отклонение = {log_cv_scores.std():.2f}")

# Случайный лес
rf_model = RandomForestClassifier(random_state=42)
rf_cv_scores = cross_val_score(rf_model, X_train, y_train, cv=5)
print(f"Случайный лес: средняя точность = {rf_cv_scores.mean():.2f}, стандартное отклонение = {rf_cv_scores.std():.2f}")

# Градиентный бустинг
gb_model = GradientBoostingClassifier(random_state=42)
gb_cv_scores = cross_val_score(gb_model, X_train, y_train, cv=5)
print(f"Градиентный бустинг: средняя точность = {gb_cv_scores.mean():.2f}, стандартное отклонение = {gb_cv_scores.std():.2f}")

# Подбор гиперпараметров для случайного леса
param_grid = {
    'n_estimators': [50, 100, 150],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10]
}

grid_search = GridSearchCV(RandomForestClassifier(random_state=42), param_grid, cv=5, scoring='accuracy')
grid_search.fit(X_train, y_train)

print("Лучшие гиперпараметры для случайного леса:", grid_search.best_params_)
print(f"Лучшая точность: {grid_search.best_score_:.2f}")


## Результаты моделирования

### Задание 8 (1 балл)
Измерьте качество итоговой модели на кросс-валидации. Выполните предсказание на тестовом множестве и сохраните их в переменную ``y_test``. Измерьте итоговое качество на тестовом множестве.

In [ ]:
# Обучение модели с оптимальными гиперпараметрами
best_rf_model = grid_search.best_estimator_

# Кросс-валидация на обучающем множестве
cv_scores_final = cross_val_score(best_rf_model, X_train, y_train, cv=5)
print(f"Итоговая модель (Случайный лес): средняя точность на кросс-валидации = {cv_scores_final.mean():.2f}, стандартное отклонение = {cv_scores_final.std():.2f}")

# Предсказание на тестовом множестве
X_test = data_test_encoded.drop('Survived', axis=1)  # Убедитесь, что y_test у вас нет в тестовом наборе данных
y_test = best_rf_model.predict(X_test)

# Оценка качества на тестовом множестве
from sklearn.metrics import accuracy_score

# Если у вас есть истинные метки y_test, измерьте качество
# Например, если y_test есть в test_data, замените test_data['Survived'] на соответствующее значение
accuracy = accuracy_score(test_data['Survived'], y_test)
print(f"Итоговая точность на тестовом множестве: {accuracy:.2f}")


## Выводы
### Задание 9 (3 балла)
Сделайте выводы. Какие из идей сработали? Какие оказались лишними?

Во-первых, выделение новых признаков, таких как длина имени (NameLength), оказалось полезным, так как этот признак может указывать на социальный статус пассажиров: более длинные имена часто принадлежат людям с титулами, что может влиять на шансы на выживание. Первая буква каюты (CabinLetter) также проявила значимость, так как местоположение каюты может быть связано с вероятностью спасения, хотя его влияние было менее выраженным. Разделение пассажиров на мальчиков и мужчин (AgeGroup) могло быть менее значительным, чем ожидалось, из-за отсутствия четких различий в предсказаниях.

Во-вторых, использование различных моделей продемонстрировало, что логистическая регрессия показала хорошие результаты, однако более сложные модели, такие как случайный лес и градиентный бустинг, лучше захватывали нелинейные зависимости в данных. Случайный лес, особенно после подбора гиперпараметров, продемонстрировал наилучшие результаты, что подтверждает его эффективность при работе с большим количеством признаков и их взаимодействиями. Хотя градиентный бустинг не был глубоко исследован, он также является сильным алгоритмом, который можно рассмотреть в будущем.

Кодирование категориальных признаков с помощью one-hot кодирования положительно сказалось на качестве предсказаний, особенно для переменных, таких как пол, класс и место посадки. Использование кросс-валидации позволило более точно оценить качество моделей и минимизировать риск переобучения. Средние значения точности и стандартное отклонение обеспечили надежную оценку.

Некоторые идеи, такие как создание возрастной группы, могли оказаться менее значимыми, поскольку они не добавили существенного улучшения в моделях по сравнению с уже существующими признаками. Классификация женщин и девочек может быть менее актуальной, так как у женщин в целом были более высокие шансы на выживание.

В заключение, выделение новых признаков и применение более сложных моделей, таких как случайный лес, привели к улучшению качества предсказаний. Тем не менее, некоторые идеи оказались менее эффективными, что подчеркивает важность тщательного анализа и выбора признаков для последующего моделирования.






